In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install folium

Obtendo coordenadas do EXIF 
Créditos (https://developer.here.com/blog/getting-started-with-geocoding-exif-image-metadata-in-python3)

Versão definitiva

In [ ]:
import folium as f
from folium import plugins
from folium.plugins import MeasureControl
from PIL.ExifTags import TAGS
from PIL import Image
from PIL.ExifTags import GPSTAGS
import ipywidgets as widgets
from IPython.display import clear_output

def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()

def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

def get_decimal_from_dms(dms, ref):

    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)

def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])

    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])

    return (lat,lon)

def coordenadas(path):
    exif = get_exif(path)
    geotags = get_geotagging(exif)
    coord = get_coordinates(geotags)
    return coord

def caminho():
  path = input('Digite o caminho da imagem: ')
  print('\n')
  nome = input('Digite o nome do ponto: ')
  print('\n')
  return path,nome

def lat_lon(coord):
  lat = str(coord[0])
  lon = str(coord[1])
  return lat,lon

def marcador():
  print('Escolha um tipo de marcador para seu ponto:\ncloud\nhome\nbuilding\nchild\nexclamation\ncircle\nanchor\nmap-pin\ncompass')
  choice2 = input('Marcador escolhido: ')
  print('\n')
  print('Escolha uma cor para o marcador:\ngreen\nred\nblue\npurple')
  choice1 = input('Cor escolhida: ')
  print('\n')
  return choice1,choice2

def structure():
  path,nome = caminho()
  coord = coordenadas(path)
  lat,lon = lat_lon(coord)
  return nome,coord,lat,lon

mapa = f.Map(location = [-15.8,-47.9],tiles = 'OpenStreetMap',zoom_start = 4)
x = int(input('Quantos pontos serão plotados: '))
answer = int(input('Você deseja realizar modificações em cada marcador ou deixar padrão? Digite (1) para modificar e (0) caso não: '))

if answer == 1:
  while x > 0:
    nome,coord,lat,lon = structure()
    choice1,choice2 = marcador()
    f.Marker([coord[0], coord[1]],popup = "Ponto: "+nome+"\nLatitude: "+lat+"\nLongitude: "+lon,icon = f.Icon(color = choice1,icon = choice2,prefix = 'fa')).add_to(mapa)
    x-=1

if answer == 0:
  while x > 0:
    nome,coord,lat,lon = structure()
    f.Marker([coord[0],coord[1]],popup = "Ponto: "+nome+"\nLatitude: "+lat+"\nLongitude: "+lon).add_to(mapa)
    x-=1

mapa.add_child(MeasureControl())
mapa

Quantos pontos serão plotados: 2
Você deseja realizar modificações em cada marcador ou deixar padrão? Digite (1) para modificar e (0) caso não: 0
Digite o caminho da imagem: /content/drive/MyDrive/Cópia de 20201203_195552.jpg


Digite o nome do ponto: 1


Digite o caminho da imagem: /content/drive/MyDrive/Cópia de 20201203_192734.jpg


Digite o nome do ponto: 2




Plotagem de pontos de um arquivo csv (OBS: O nome deve ser obrigatoriamente uma string)

In [ ]:
import pandas as pd
import folium as f

path = input('Digite o caminho do arquivo: ')

df = pd.read_csv(path)

mapa2 = f.Map(location = [-15.8,-47.9],tiles = 'OpenStreetMap',zoom_start = 4)

nome = input('Digite o nome da coluna que tem o nome dos pontos: ')
lat = input('Digite o nome da coluna das Latitudes: ')
lon = input('Digite o nome da coluna das Longitudes: ')


for index, row in df.iterrows():
  folium.Marker([row[lat], row[lon]], popup=row[nome],icon=folium.Icon(icon='cloud')).add_to(mapa2)

mapa2

Digite o caminho do arquivo: /content/drive/MyDrive/Colab Notebooks/RBMC.csv
Digite o nome da coluna que tem o nome dos pontos: FID
Digite o nome da coluna das Latitudes: LATITUDE
Digite o nome da coluna das Longitudes: LONGITUDE
